In [1]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None

## Load excel file

In [2]:
kbr_data_path = '../../data/raw/KBR/'
kbr_dir_path = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file_path = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data_path, kbr_dir_path, kbr_file_path), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [3]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

## Dataframe filtering by selected columns + mapping age_bracket and sex values + some processing

In [4]:
chosen_cols = [
    'IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza)',
    'IDENTYFIKACJA OSOBY_Nr. osoby w gosp. dom.',
    'IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Godzina rozpoczęcia podróży\n[gg:mm:ss]',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Z jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca [Inne]',
    '23:59:00_motywacje'
]

data_filtered = data[chosen_cols]

data_filtered.columns = [
    'id', 
    'household_person_id',
    'questionnaire_date',
    'age_bracket', 
    'sex',
    'trip_start_time',
    'source', 
    'destination',
    'what_inne',
    'source-destination'
]

data_filtered.fillna(0, inplace=True)
# data_filtered.dropna(inplace=True)
# data_filtered.reset_index(drop=True, inplace=True)

In [5]:
age_bracket_map_dict = {
    '6-15 (dzieci)': '6-15',
    '16-19 (młodzież)': '16-19',
    '20-24 (wiek studencki)': '20-24',
    '25-44 (młodsi pracownicy)': '25-44',
    '45-60 (starsi pracownicy kobiety)': '45-60',  # '45-60(K)',
    '45-65 (starsi pracownicy mężczyźni)': '45-65',  # '45-65(M)',
    '61 i więcej (emeryci kobiety)': '61-x',  # '61-(K)',
    '66 i więcej (emeryci mężczyźni)': '66-x'  # '66-(M)'
}
data_filtered.replace({'age_bracket': age_bracket_map_dict}, inplace=True)

sex_map_dict = {
    'Kobieta': 'K',
    'Mężczyzna': 'M'
}
data_filtered.replace({'sex': sex_map_dict}, inplace=True)

data_filtered['age_sex_comb'] = data_filtered['age_bracket'] + '_' + data_filtered['sex']
data_filtered['person_id'] = data_filtered['id'] + data_filtered['questionnaire_date'].astype('str') + data_filtered['household_person_id'].astype('str')
data_filtered['trip_start_time'] = data_filtered['trip_start_time'].astype('str')
data_filtered['source'] = data_filtered['source'].str.replace('Inne', 'inne')
data_filtered['destination'] = data_filtered['destination'].str.replace('Inne', 'inne')
data_filtered['what_inne'] = data_filtered['what_inne'].str.replace('Inne', 'inne')
data_filtered['source-destination'] = data_filtered['source-destination'].str.replace('Inne', 'inne')

## Distribution: number of travels grouped by combinations of age group and gender

### Prepare data

In [6]:
len(data_filtered['person_id'].unique())

6872

In [7]:
data_grouped = data_filtered.groupby(['person_id', 'age_sex_comb'])['person_id'].count().reset_index(name='travel_amount')

data_grouped.shape[0]

7304

In [8]:
data_grouped.head()

,person_id,age_sex_comb,travel_amount
0,LOS_100-100-13052018-06-141.0,61-x_K,2
1,LOS_100-100-1642018-06-221.0,25-44_K,1
2,LOS_100-100-1642018-06-221.0,45-60_K,1
3,LOS_100-100-22812018-06-221.0,25-44_K,2
4,LOS_100-100-482018-06-221.0,20-24_K,2


In [9]:
data_grouped = data_grouped.groupby(['age_sex_comb', 'travel_amount'])['travel_amount'].count().reset_index(name='count')

data_grouped.shape[0]

74

In [10]:
data_grouped.head()

,age_sex_comb,travel_amount,count
0,16-19_K,1,22
1,16-19_K,2,69
2,16-19_K,3,4
3,16-19_K,4,6
4,16-19_K,5,1


### Distribution

In [11]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
max_travel_amount = 6   # df_grouped['travel_amount'].max()

travel_amount_distr = {}

for age_sex_comb in unique_age_sex_comb:
    travel_amount_distr[age_sex_comb] = {}

    for travel_amount in range(1, max_travel_amount+1):
        try:
            count = int(data_grouped[
                (data_grouped['travel_amount'] == travel_amount) & 
                (data_grouped['age_sex_comb'] == age_sex_comb)
            ]['count'])
        except TypeError:
            count = 1

        travel_amount_distr[age_sex_comb][travel_amount] = count

In [12]:
travel_amount_distr

{'16-19_K': {1: 22, 2: 69, 3: 4, 4: 6, 5: 1, 6: 1},
 '16-19_M': {1: 14, 2: 104, 3: 8, 4: 6, 5: 1, 6: 2},
 '20-24_K': {1: 49, 2: 186, 3: 30, 4: 23, 5: 1, 6: 3},
 '20-24_M': {1: 63, 2: 179, 3: 20, 4: 23, 5: 2, 6: 4},
 '25-44_K': {1: 222, 2: 765, 3: 84, 4: 76, 5: 8, 6: 9},
 '25-44_M': {1: 196, 2: 764, 3: 70, 4: 46, 5: 8, 6: 4},
 '45-60_K': {1: 180, 2: 532, 3: 49, 4: 35, 5: 8, 6: 1},
 '45-65_M': {1: 201, 2: 638, 3: 41, 4: 39, 5: 3, 6: 1},
 '6-15_K': {1: 67, 2: 274, 3: 10, 4: 23, 5: 1, 6: 1},
 '6-15_M': {1: 43, 2: 225, 3: 10, 4: 11, 5: 1, 6: 1},
 '61-x_K': {1: 413, 2: 692, 3: 23, 4: 36, 5: 5, 6: 1},
 '66-x_M': {1: 266, 2: 360, 3: 17, 4: 18, 5: 2, 6: 1}}

### Probability distribution

In [13]:
for age_sex_comb, travel_amounts in travel_amount_distr.items():
    travel_amounts_sum = np.array(list(travel_amounts.values())).sum()
    if travel_amounts_sum != 0:
        for i in list(travel_amount_distr[age_sex_comb].keys()):
            travel_amount_distr[age_sex_comb][i] = travel_amount_distr[age_sex_comb][i] / travel_amounts_sum

In [14]:
for age_sex_comb, travel_amounts in travel_amount_distr.items():
    print(age_sex_comb, end='\t -  ')
    print(travel_amounts)
    
# travel_amount_distr

16-19_K	 -  {1: 0.21359223300970873, 2: 0.6699029126213593, 3: 0.038834951456310676, 4: 0.05825242718446602, 5: 0.009708737864077669, 6: 0.009708737864077669}
16-19_M	 -  {1: 0.1037037037037037, 2: 0.7703703703703704, 3: 0.05925925925925926, 4: 0.044444444444444446, 5: 0.007407407407407408, 6: 0.014814814814814815}
20-24_K	 -  {1: 0.1678082191780822, 2: 0.636986301369863, 3: 0.10273972602739725, 4: 0.07876712328767123, 5: 0.003424657534246575, 6: 0.010273972602739725}
20-24_M	 -  {1: 0.21649484536082475, 2: 0.6151202749140894, 3: 0.06872852233676977, 4: 0.07903780068728522, 5: 0.006872852233676976, 6: 0.013745704467353952}
25-44_K	 -  {1: 0.19072164948453607, 2: 0.6572164948453608, 3: 0.07216494845360824, 4: 0.06529209621993128, 5: 0.006872852233676976, 6: 0.007731958762886598}
25-44_M	 -  {1: 0.1801470588235294, 2: 0.7022058823529411, 3: 0.06433823529411764, 4: 0.042279411764705885, 5: 0.007352941176470588, 6: 0.003676470588235294}
45-60_K	 -  {1: 0.2236024844720497, 2: 0.660869565217

### Save probability distribution to .json

In [15]:
out_dir = '../../data/processed/mc/travel_planning'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [16]:
out_file = 'travel_amount_distr.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(travel_amount_distr, f)

## Distribution: travel purpose grouped by combinations (age group + gender) and type of starting place

### Prepare data

In [17]:
data_grouped = data_filtered.groupby(['age_sex_comb', 'source', 'destination'])['destination'].count().reset_index(name='count')

data_grouped.shape[0]

171

In [18]:
data_grouped.head()

,age_sex_comb,source,destination,count
0,16-19_K,dom,inne,18
1,16-19_K,dom,praca,6
2,16-19_K,dom,szkoła,56
3,16-19_K,dom,uczelnia,12
4,16-19_K,inne,dom,17


In [19]:
# data_filtered[
#     (data_filtered['age_sex_comb'] == '16-19_K') &
#     (data_filtered['source'] == 'dom')
# ]['destination'].value_counts()

In [20]:
data_grouped['source'] = data_grouped['source'].str.replace('szkoła', 'szkola')
data_grouped['destination'] = data_grouped['destination'].str.replace('szkoła', 'szkola')

In [21]:
data_grouped.head()

,age_sex_comb,source,destination,count
0,16-19_K,dom,inne,18
1,16-19_K,dom,praca,6
2,16-19_K,dom,szkola,56
3,16-19_K,dom,uczelnia,12
4,16-19_K,inne,dom,17


### Distribution

In [22]:
unique_age_sex_comb = data_grouped['age_sex_comb'].unique()
unique_place_types = sorted(data_grouped['source'].unique())

travel_purpose_distr = {}

for age_sex_comb in unique_age_sex_comb:
    travel_purpose_distr[age_sex_comb] = {}

    for source_place_type in unique_place_types:
        travel_purpose_distr[age_sex_comb][source_place_type] = {}

        for destination_place_type in unique_place_types:
            try:
                count = int(data_grouped[
                    (data_grouped['age_sex_comb'] == age_sex_comb) &
                    (data_grouped['source'] == source_place_type) &
                    (data_grouped['destination'] == destination_place_type)
                ]['count'])
            except TypeError:
                count = 1

            travel_purpose_distr[age_sex_comb][source_place_type][destination_place_type] = count

In [23]:
for age_sex_comb, source_place_types in travel_purpose_distr.items():
    for source_place_type, destination_place_types in source_place_types.items():
        print(age_sex_comb, end='  -  ')
        print(source_place_type, end='  \t')
        print(destination_place_types)

# travel_purpose_distr

16-19_K  -  dom  	{'dom': 1, 'inne': 18, 'praca': 6, 'szkola': 56, 'uczelnia': 12}
16-19_K  -  inne  	{'dom': 17, 'inne': 6, 'praca': 1, 'szkola': 2, 'uczelnia': 1}
16-19_K  -  praca  	{'dom': 5, 'inne': 1, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_K  -  szkola  	{'dom': 55, 'inne': 3, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_K  -  uczelnia  	{'dom': 11, 'inne': 1, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_M  -  dom  	{'dom': 1, 'inne': 22, 'praca': 5, 'szkola': 92, 'uczelnia': 12}
16-19_M  -  inne  	{'dom': 31, 'inne': 5, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_M  -  praca  	{'dom': 4, 'inne': 1, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_M  -  szkola  	{'dom': 83, 'inne': 7, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
16-19_M  -  uczelnia  	{'dom': 11, 'inne': 1, 'praca': 1, 'szkola': 1, 'uczelnia': 1}
20-24_K  -  dom  	{'dom': 2, 'inne': 82, 'praca': 53, 'szkola': 12, 'uczelnia': 135}
20-24_K  -  inne  	{'dom': 110, 'inne': 26, 'praca': 1, 'szkola': 1, 'uczelnia': 

### Probability distribution

In [24]:
for age_sex_comb, source_place_types in travel_purpose_distr.items():
    for source_place_type, destination_place_types in source_place_types.items():
        count_sum = np.array(list(destination_place_types.values())).sum()
        if count_sum != 0:
            for i in list(travel_purpose_distr[age_sex_comb][source_place_type].keys()):
                travel_purpose_distr[age_sex_comb][source_place_type][i] = travel_purpose_distr[age_sex_comb][source_place_type][i] / count_sum

In [25]:
# travel_purpose_distr

for age_sex_comb, source_place_types in travel_purpose_distr.items():
    for source_place_type, destination_place_types in source_place_types.items():
        print(age_sex_comb, end='  -  ')
        print(source_place_type, end='  \t')
        print(destination_place_types)

16-19_K  -  dom  	{'dom': 0.010752688172043012, 'inne': 0.1935483870967742, 'praca': 0.06451612903225806, 'szkola': 0.6021505376344086, 'uczelnia': 0.12903225806451613}
16-19_K  -  inne  	{'dom': 0.6296296296296297, 'inne': 0.2222222222222222, 'praca': 0.037037037037037035, 'szkola': 0.07407407407407407, 'uczelnia': 0.037037037037037035}
16-19_K  -  praca  	{'dom': 0.5555555555555556, 'inne': 0.1111111111111111, 'praca': 0.1111111111111111, 'szkola': 0.1111111111111111, 'uczelnia': 0.1111111111111111}
16-19_K  -  szkola  	{'dom': 0.9016393442622951, 'inne': 0.04918032786885246, 'praca': 0.01639344262295082, 'szkola': 0.01639344262295082, 'uczelnia': 0.01639344262295082}
16-19_K  -  uczelnia  	{'dom': 0.7333333333333333, 'inne': 0.06666666666666667, 'praca': 0.06666666666666667, 'szkola': 0.06666666666666667, 'uczelnia': 0.06666666666666667}
16-19_M  -  dom  	{'dom': 0.007575757575757576, 'inne': 0.16666666666666666, 'praca': 0.03787878787878788, 'szkola': 0.696969696969697, 'uczelnia':

### Save probability distribution to .json

In [26]:
out_dir = '../../data/processed/mc/travel_planning'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [27]:
out_file = 'travel_purpose_distr.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(travel_purpose_distr, f)